In [1]:
# https://people.brandeis.edu/~blebaron/classes/fin250a/regression/armax.html
library(forecast)
library(tidyverse) # tidy data

library(tseries)
library(MTS)

library(tsDyn)

library(vars) # post-estimation procedures to ensure adequacy of model
library(urca)
library(mFilter)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: strucchange

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: sandwich


Attaching package: ‘strucchange’


The following object is masked from ‘package:stringr’:

    boundary


Loading required package: urca

Loading required package: lm

# Lagged Timeseries Analysis
- the cross-correlation
- the leading variable
- the lag

# VECM
For the sake of demonstration purposes, one county selected at random will be used to demonstrate these concepts.

In [2]:
# generate a random number between 1 and 817
sample(0:1191, 1) #400

[1] 1021

# VECM (Vector Error Correction Method)
The Coaint-Johansen test tests for co-integration. This would indicate that a linear combination of non-stationary variables could potentially be stationary. Co-integration is determined using the Coaint-Johansen or the Engle-Granger tests. These tests are applied to the original timeseries, rather than to a timeseries first difference. 
The Coaint-Johansen test or test for co-integration seeks to identify if long-run relationships exist between series, and if so, identifies how many there are. If there is a co-integrating relationship, construct a VECM.VECM produces more efficient coefficient estimates than the next process, VAR. 

In [3]:
ts_data <- read_csv('employment_covid_mobility_ts.csv')
# drop X column
drops <- c("X")
ts_data <- ts_data[ , !(names(ts_data) %in% drops)]

New names:
* `` -> ...1

Rows: 117711 Columns: 6

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (5): ...1, countyfips, emp_incbelowmed, new_case_rate, gps_away_from_home
date (1): date


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [4]:
ts_data$countyfips <- as.character(ts_data$countyfips)

Pivot the data.

In [5]:
pivot_emp <- ts_data %>%
    dplyr::select(date, countyfips, emp_incbelowmed) %>%
        pivot_wider(names_from = countyfips, values_from = emp_incbelowmed)

In [6]:
dim(pivot_emp) # 99 timesteps and 1190 counties

[1]   99 1190

In [7]:
pivot_covid <- ts_data %>%
    dplyr::select(date, countyfips, new_case_rate) %>%
        pivot_wider(names_from = countyfips, values_from = new_case_rate)

In [8]:
dim(pivot_covid) # 99 timesteps and 1190 counties

[1]   99 1190

In [9]:
pivot_mobility <- ts_data %>%
    dplyr::select(date, countyfips, gps_away_from_home) %>%
        pivot_wider(names_from = countyfips, values_from = gps_away_from_home)

## convert to timeseries

In [10]:
pivot_emp_ts <- data.frame(apply(pivot_emp[,2:1190], 2, function(x) ts(x)), check.names=FALSE)
pivot_emp_ts$date <- pivot_emp$date

In [11]:
pivot_covid_ts <- data.frame(apply(pivot_covid[,2:1085], 2, function(x) ts(x)),check.names=FALSE)
pivot_covid_ts$date <- pivot_covid$date

In [12]:
pivot_mobility_ts <- data.frame(apply(pivot_mobility[,2:1085], 2, function(x) ts(x)),check.names=FALSE)
pivot_mobility_ts$date <- pivot_mobility$date

## choose county at random

In [13]:
#choose random county and store emp timeseries to emp
emp <- pivot_emp[,1028]
#colnames(emp)[1] <- c("emp")

In [15]:
#choose random county and store covid timeseries to covid19
covid19 <- pivot_covid[,1028]
colnames(covid19)[1] <- c("covid")

In [16]:
#choose random county and store covid timeseries to covid19
mobility <- pivot_mobility[,1028]
colnames(mobility)[1] <- c("mobility")

## bind to one object

In [17]:
dset <- cbind(emp, covid19, mobility)

# test for lag selection // determine lag length
 - use VARselect
 - estimate VAR in levels and choose lag specification that minimizes the information criterion
 
## Method 1

In [35]:
# lag selection criteria
lagSelect <- VARselect(dset, lag.max = 30, type = "const")
lagSelect$selection # 3 was chosen most often

Warning message in log(sigma.det):
“NaNs produced”
Warning message in log(sigma.det):
“NaNs produced”
Warning message in log(sigma.det):
“NaNs produced”


AIC(n)  HQ(n)  SC(n) FPE(n) 
    23     23     23     22

## Method 2

In [33]:
# Estimate VAR
var_aic <- vars::VAR(dset, lag.max = 30, type = "const", ic = "AIC", season = NULL)

# Lag order suggested by AIC
var_aic$p

Warning message in log(sigma.det):
“NaNs produced”
Warning message in log(sigma.det):
“NaNs produced”
Warning message in log(sigma.det):
“NaNs produced”


AIC(n) 
    23

The first method may be preferred as many tests are applied. The lag occuring most often is the selected lag. Here, the VEC model corresponding to the VAR in levels has 23 lags.

# specify the maximum likelihood estimator of the Johanssen test
1) uses the trace statistic approach
2) uses the maximum eigenvalue
Estimating the Vector Error Correction requires the inclusion of deterministic terms. A common strategy is to add a linear term to the error correction term and add a constant to the non-cointegration part of the equation. 

### without linear trend

In [19]:
ctest.trace <- urca::ca.jo(dset, type ='trace', ecdet = "const", K = 22, season = NULL) # accounts for model intercepts
# calculates rank of cointegration matrix rs
summary(ctest.trace) 


###################### 
# Johansen-Procedure # 
###################### 

Test type: trace statistic , without linear trend and constant in cointegration 

Eigenvalues (lambda):
[1]  7.897248e-01  5.183679e-01  1.036293e-01 -7.494005e-16

Values of teststatistic and critical values of test:

           test 10pct  5pct  1pct
r <= 2 |   8.42  7.52  9.24 12.97
r <= 1 |  64.68 17.85 19.96 24.60
r = 0  | 184.75 32.00 34.91 41.07

Eigenvectors, normalised to first column:
(These are the cointegration relations)

               X48479.l22     covid.l22 mobility.l22     constant
X48479.l22    1.000000000  1.0000000000   1.00000000  1.000000000
covid.l22    -0.001736248 -0.0005309413  -0.00972061 -0.009772328
mobility.l22 -0.979763641  1.6867065052   2.48095340  3.356131944
constant      0.279624965  0.4143357467   0.52077718  0.560343007

Weights W:
(This is the loading matrix)

              X48479.l22    covid.l22  mobility.l22      constant
X48479.d     -0.05270173   0.06787369   -0.034500

The values of test statistic and critical values of test table pertain to critical values at the 10%, 5% and 1% significance levels. As is typical, we will use a critical value of 5%.
r represents the route of your error correction terms matrix and pertains to the number of cointegrating relationships that exist. We find the trace test statistics in the first column.
 - When r=0, the test statistic is greater than the value specified at the 5% level (34.91). This suggests that we reject the null and there is at least one (1) cointegrating relationship.
 - When r<=1, the test statistic is greater than the value specified at the 5% level (19.96). This suggests that we reject the null and there are at least two (2) cointegrating relationships.
 - When r<=2, the test statistic is less than the value specified at the 5% level (9.24). This suggests that we fail to reject the null.
 
There are therefore, at least two (2) cointegrating relationships in this system.

In [20]:
ctest.eigen <- ca.jo(dset, type ='eigen', ecdet = "const", K = 22) # accounts for model intercepts
summary(ctest.eigen)


###################### 
# Johansen-Procedure # 
###################### 

Test type: maximal eigenvalue statistic (lambda max) , without linear trend and constant in cointegration 

Eigenvalues (lambda):
[1]  7.897248e-01  5.183679e-01  1.036293e-01 -7.494005e-16

Values of teststatistic and critical values of test:

           test 10pct  5pct  1pct
r <= 2 |   8.42  7.52  9.24 12.97
r <= 1 |  56.25 13.75 15.67 20.20
r = 0  | 120.07 19.77 22.00 26.81

Eigenvectors, normalised to first column:
(These are the cointegration relations)

               X48479.l22     covid.l22 mobility.l22     constant
X48479.l22    1.000000000  1.0000000000   1.00000000  1.000000000
covid.l22    -0.001736248 -0.0005309413  -0.00972061 -0.009772328
mobility.l22 -0.979763641  1.6867065052   2.48095340  3.356131944
constant      0.279624965  0.4143357467   0.52077718  0.560343007

Weights W:
(This is the loading matrix)

              X48479.l22    covid.l22  mobility.l22      constant
X48479.d     -0.0527017

The conclusion is the same as there appear to be at least two (2) cointegrating relationships in this system.

### with linear trend

In [34]:
# Estimate
ctest.linear <- ca.jo(dset, ecdet = "none", type = "trace", K = 23, season = NULL)
summary(ctest.linear)


###################### 
# Johansen-Procedure # 
###################### 

Test type: trace statistic , with linear trend 

Eigenvalues (lambda):
[1] 0.8781720 0.6040655 0.1144544

Values of teststatistic and critical values of test:

           test 10pct  5pct  1pct
r <= 2 |   9.24  6.50  8.18 11.65
r <= 1 |  79.65 15.66 17.95 23.52
r = 0  | 239.64 28.71 31.52 37.22

Eigenvectors, normalised to first column:
(These are the cointegration relations)

               X48479.l23    covid.l23 mobility.l23
X48479.l23    1.000000000  1.000000000  1.000000000
covid.l23    -0.001470783 -0.007220507 -0.008203121
mobility.l23 -0.840617173 -8.545106378  2.059954864

Weights W:
(This is the loading matrix)

              X48479.l23    covid.l23 mobility.l23
X48479.d       0.5659127  -0.01930686  -0.02463982
covid.d    11641.5117690 204.43696823 174.34264202
mobility.d    -0.1895023  -0.16940342   0.18087481


### We read this matrix result in much the same way. Here, our results differ in that this matrix seems to suggest that three (3) co-integrating relationships are present.

## Build VECM

In [21]:
model = VECM(dset, 22, r=2, estim=("ML")) # specify data, lags, cointegrating relationships

In [22]:
summary(model)

Warning message in if (class(x) == "numeric") return(noquote(r)):
“the condition has length > 1 and only the first element will be used”
Warning message in if (class(x) == "matrix") return(matrix(noquote(r), ncol = ncol(x), :
“the condition has length > 1 and only the first element will be used”
Warning message in if (class(x) == "numeric") return(noquote(r)):
“the condition has length > 1 and only the first element will be used”
Warning message in if (class(x) == "matrix") return(matrix(noquote(r), ncol = ncol(x), :
“the condition has length > 1 and only the first element will be used”


#############
###Model VECM 
#############
Full sample size: 99 	End sample size: 76
Number of variables: 3 	Number of estimated slope parameters 207
AIC -2023.384 	BIC -1536.261 	SSR 140.0888
Cointegrating vector (estimated by ML):
   48479 covid    mobility
r1     1     0    1.130194
r2     0     1 1339.975059


                  ECT1                     ECT2               
Equation 48479    0.5466(0.1647)*          -0.0007(0.0003)*   
Equation covid    11845.9354(2026.8513)*** -18.5982(3.1775)***
Equation mobility -0.3589(1.1205)          0.0015(0.0018)     
                  Intercept              48479 -1                 
Equation 48479    0.1629(0.0469)*        -1.0990(0.2625)**        
Equation covid    3302.5072(576.8394)*** -2152.4978(3229.5914)    
Equation mobility -0.0380(0.3189)        1.1911(1.7855)           
                  covid -1            mobility -1            
Equation 48479    0.0007(0.0002)*     0.2502(0.1858)         
Equation covid    17.4749(2.9400)***  95

Indicates a small, positive correlation between covid and emp, and a big positive relationship between mobility and covid.*** results can vary based on order of variables
The model summary also logs the Error Correction Term (ECT), the intercept and each variable at various lags (**emp - 1** indicates emp at lag 1). And asterik* next to the lag indicates significance. 
The most significant variables and their lags indicate mostly long-run relationships and include variables at: 
 - emp - 11
 - covid - 10
 - covid - 8
 - covid - 6
 - emp - 5
 - covid - 4
 - covid - 3 
 - and covid - 2

This indicates a delayed relationship for the indicated variables and their value at the indicated lags.

# convert to VAR -- move this to VAR notebook

In [24]:
VARmodel <- vec2var(ctest.trace, r=2)

In [25]:
serial.test(VARmodel, lags.pt = 22, type = 'PT.asymptotic')


	Portmanteau Test (asymptotic)

data:  Residuals of VAR object VARmodel
Chi-squared = 285.62, df = 3, p-value < 2.2e-16

$serial

	Portmanteau Test (asymptotic)

data:  Residuals of VAR object VARmodel
Chi-squared = 285.62, df = 3, p-value < 2.2e-16



In [ ]:
# arch effects would indicate clustered volatility in the model